# PIP-PRoS: Controlo do atuador linear

Neste notebook serão introduzidos a classe criada para o controle do Stepper Motor utilizado no atuador linear e os métodos para o seu controlo.

Para mais informações, acesse [o site do projeto](https://sites.google.com/tecnico.ulisboa.pt/pip-pros/)!

## Bibliotecas utilizadas

Como o controlo do atuador se dá via Serial por meio de um Arduino, será utilizada a biblioteca `serial`.

A biblioteca `time` será usada para introduzir delays após a chamada de certas funções.

In [23]:
import serial 
import time

## Introdução ao sistema desenvolvido

Como descrito no manual do projeto, que pode ser encontrado no website, foi criado um protocolo para a comunicação via serial com o Arduino. O Arduino interpreta os comandos e gera os sinais necessários para que o driver do motor o controle.

Para facilitar o uso da interface, foi criada a classe `SerialStepperMotor` que será explicada em sequência.

## Classe `SerialStepperMotor`

In [ ]:
class SerialStepperMotor:
    def __init__(self, port, baudrate):
        self.port = port
        self.baudrate = baudrate
        self.serial_connection = serial.Serial(self.port, self.baudrate, timeout=1)
        time.sleep(1)

    def is_connected(self):
        self.serial_connection.write(bytearray("c 200", 'ascii'))
        for _ in range(10):
            answer = self.serial_connection.readline()
            if answer[0:3] == b'200':
                return True
            time.sleep(0.5)
        else:
            return False
    
    def calibrate(self):
        print("Starting calibration...")
        self.serial_connection.write(bytearray("i", 'ascii'))
        calibration_finished = False
        while not calibration_finished:
            answer = self.serial_connection.readline()
            if answer[0:3] == b'201':
                calibration_finished = True
        print("Calibrated...")

    def close(self):
        self.serial_connection.close()

    def move_to(self, position_percentage):
        """
        Receives position in percentage and moves the motor 
        to the equivalent percentage of its full trajectory.
        """
        self.serial_connection.write(bytearray("m " + str(position_percentage) +" \n", 'ascii'))
        print("Sent: m " + str(position_percentage))

    def stop(self):
        self.serial_connection.write(bytearray("p \n", 'ascii'))
        print("Sent: p")

A classe dispõe apenas dos métodos essenciais para realizar a interface entre o Arduino e o programa `main` em python. O protocolo como implementado no arquivo `linear_actuator_controller` é mais compreensivo, com comandos de ``set`` e ``get`` da velocidade e aceleração do atuador, que podem ser usados via Monitor Serial. 

A instanciação de um objeto da classe SerialStepperMotor realiza a conexão com o sistema do atuador linear:

In [ ]:
motor = SerialStepperMotor("COM8", 115200)

## Verificação do status da conexão

A estratégia utilizada para verificar a validade da conexão é simples: é enviado via Serial uma mensagem para o controlador, quando esse o recebe envia a mensagem de volta. Caso o computador receba essa mensagem, a conexão é considerada válida. Para verificar o status da conexão basta usar o método ``motor.is_connected()``.

In [ ]:
print(motor.is_connected())

## Calibração do motor

No estado atual, o sistema não conhece a posição do sistema de bloqueio da bola. A estratégia escolhida para conhecer sua posição, assim como é comumente feito para sistemas de impressoras 3D, foi o uso de fins-de-curso. Os fins-de-curso são apenas switches localizados nas extremidades do perfil acionados com o movimento do sistema de bloqueio da bola.

É importante notar, antes, como um StepperMotor lida com posições. A posição de um motor de passos para a biblioteca de controle usada (AccelStepper) é calculada como a quantidade de passos desde a conexão com o motor. Ou seja, a posição 0 é a posição inicial do motor, 2000 seria a posição a 2000 passos para um lado e -2000 seria a posição para o outro lado.

Assim, para fazer a calibração, o motor é movimentado a velocidades baixas na direção de um dos fins-de-curso até este ser acionado. Quando é, guarda a posição atual como o primeiro limite e inverte sua velocidade, dirigindo o sistema ao outro switch. Quando o atinge, ambos os limites do campo são conhecidos e é possível enviar o motor para a posição correspondente ao meio do campo ((upper_limit - lower_limit)/2).

Esse processo pode ser realizado por meio do método ``motor.calibrate()``:

In [ ]:
motor.calibrate()

## Movimentação do sistema de bloqueio ao longo do campo

Para movimentar o robô basta indicar em qual ponto da trajetória completa (0.0 - 1.0) o robô deve estar, sendo 0 o limite inferior (na visão da câmera) e 1 o limite superior. Para posicioná-lo a X% da trajetória, basta enviar X/100:

In [ ]:
# Envia para o limite inferior
motor.move_to(0)
time.sleep(1)

# Envia para o limite superior
motor.move_to(1)
time.sleep(1)

# Envia para o meio do campo
motor.move_to(0.5)
time.sleep(1)

# Envia para o quarto superior da trajetória
motor.move_to(0.75)
time.sleep(1)

## Interrupção do movimento

Na situação em que o setpoint é atualizado durante o movimento do motor, é interessante contar com a possibilidade de parar o seu movimento e iniciar um outro. Para isso foi implementado o método ``motor.stop()``, de uso simples:

In [ ]:
# Envia o motor para o limite sueprior
motor.move_to(1)
time.sleep(1)

# Tenta enviar para o limite inferior, mas o movimento é interrompido
motor.move_to(0)
time.sleep(0.01)
motor.stop()

## Fechar a conexão com o sistema

Por último, é sempre necessário, ao fim do programa, desconectar o sistema:

In [ ]:
motor.close()